In [ ]:
import fastdyn_fic_dmf as dmf
import numpy as np
from scipy.io import loadmat

C = loadmat('../data/DTI_fiber_consensus_HCP.mat')['connectivity'][:200, :200]
C = 0.2*C/np.max(C)
N = C.shape[0]
params = dmf.default_params(C=C)

# Homeostatic parameters depend on the objective rate
params['obj_rate'] = 3.44
# We have pre-computed the a,b coefficients for objective rates 3.44 
if params['obj_rate'] == 3.44:
    fit_res = np.load("../data/fit_res_3-44.npy")
    b = fit_res[0] # First element is the slope
    a = fit_res[1]
else:
    # Throw an exception stating that you need to calculate a,b coefficients for the desired objective rate
    Exception("Need to calculate a,b coefficients for the desired objective rate")    

# To explore the effect of the decay this can be turned off
# In general, if we want plasticity we use both set decay and plasticity as True
params["with_decay"] = True
params["with_plasticity"] = True
LR = 10
# We use the a,b coefficients to calculate the decay
DECAY = np.exp(a+np.log(LR)*b)
# To allow for heterogeneities in the decay and learning rate we can set them as vectors
# If not, simply set them as scalars
params['taoj_vector'] =  np.ones(N)*DECAY
params['lr_vector'] = np.ones(N)*LR

# Set the global coupling strength
G_VAL = 2.1
params['G'] = G_VAL
# We use the linear solution from fastDMF paper to initialize FIC values
params['J'] = 0.75*params['G']*params['C'].sum(axis=0).squeeze() + 1


# Short simulation
nb_steps = 50000
params['seed'] = 1

# Assume you want everyhing returned
params["return_bold"] = True
params["return_fic"] = True
params["return_rate"] = True
# If not it will return the last batch of values calcualted

rates_dyn, rates_inh_dyn, bold_dyn, fic_t_dyn = dmf.run(params, 50000)
